In [1]:
#1. imports
import os
import pandas as pd
import scipy.io as sio
import glob
import re
import numpy as np
import time
import matplotlib.pyplot as plt

from pyimzml.ImzMLParser import ImzMLParser

In [2]:
#2. paths and parameters
DataPath = 'n:/'

In [3]:
#get all the relevant folders
folders = []
TMA_IDs = []
for r, d, f in os.walk(DataPath):
    for folder in d:
        if 'Patient' in folder:
            folders.append(r+'/'+folder + '/')
            Pos1 = r.find('ECTMA',0,100)
            Pos2 = r.find('imzml',0,100)
            TMA_ID = r[Pos1+15:Pos2-11]
            TMA_IDs.append(TMA_ID)     
SourceData_df = pd.DataFrame()
SourceData_df['TMA_ID'] = TMA_IDs
SourceData_df['TMA_path'] = folders

In [4]:
#load an explicit mapping between patient and core IDs
CorePatientMap_df = pd.read_excel('../MetaData/TMA 16-006 map_nodata.xlsx',sheet_name='Core ID')
CorePatientMap_df=CorePatientMap_df[['Core ID','Endo Patient ID']]

In [5]:
#get the labels for positive/negative etc
LabelsB_df =  pd.read_excel('../MetaData/TMA core ID.xlsx',sheet_name='TMA B')
LabelsD_df =  pd.read_excel('../MetaData/TMA core ID.xlsx',sheet_name='TMA D').iloc[:,0:3]
LabelsE_df =  pd.read_excel('../MetaData/TMA core ID.xlsx',sheet_name='TMA E')
PerPatientLabels_df = pd.concat((LabelsB_df,LabelsD_df,LabelsE_df))
print(LabelsB_df.shape,LabelsD_df.shape,LabelsE_df.shape)
aa=list(PerPatientLabels_df.columns)
aa[1]='Patient ID'
aa[2]='LNM Label'
PerPatientLabels_df.columns=aa
UniquePatientsInBDE=np.unique(PerPatientLabels_df['Patient ID'])
print('Number of unique patients in B,D and E',UniquePatientsInBDE.shape[0])
print(' ')
print(PerPatientLabels_df['LNM Label'].value_counts(dropna=False),'but 491, negative, is counted twice')
print(' ')
#keep only positive and negative
PerPatientLabels_df=PerPatientLabels_df[PerPatientLabels_df['LNM Label'].isin(['positive','negative'])]
PerPatientLabels_df=PerPatientLabels_df.reset_index(drop=True)
print('Num Patients = ',PerPatientLabels_df.shape[0])

(122, 3) (128, 3) (53, 3)
Number of unique patients in B,D and E 302
 
negative      215
positive       43
not tested     41
NaN             4
Name: LNM Label, dtype: int64 but 491, negative, is counted twice
 
Num Patients =  258


In [6]:
#now create a per-core dataframe with only the patients in the above data frame
CoreIndexed_PatientMap_df=CorePatientMap_df.set_index('Core ID',drop=True)
CoreIndexed_PatientMap_df['LNM Label']=''
CoreIndexed_PatientMap_df['PerPatientTMACoreID']=''
CoreIndexed_PatientMap_df['PerPatientTMAPatientID']=''
for i in CoreIndexed_PatientMap_df.index:
    PatientID = CoreIndexed_PatientMap_df.at[i,'Endo Patient ID']
    Label = PerPatientLabels_df[PerPatientLabels_df['Patient ID']==PatientID]['LNM Label'].values
    PerPatientTMACoreID = PerPatientLabels_df[PerPatientLabels_df['Patient ID']==PatientID]['TMA core ID'].values
    PerPatientTMAPatientID = PerPatientLabels_df[PerPatientLabels_df['Patient ID']==PatientID]['Patient ID'].values
    if len(Label)==1:
        CoreIndexed_PatientMap_df.at[i,'LNM Label']=Label[0]
        CoreIndexed_PatientMap_df.at[i,'PerPatientTMACoreID']=PerPatientTMACoreID[0]
        CoreIndexed_PatientMap_df.at[i,'PerPatientTMAPatientID']=PerPatientTMAPatientID[0]
    elif len(Label) > 1:
        #two of the cases appear in separate rows, so handle this for core IDs 510 and 691
        if i ==510:
            CoreIndexed_PatientMap_df.at[i,'LNM Label']=Label[0]
            CoreIndexed_PatientMap_df.at[i,'PerPatientTMACoreID']=PerPatientTMACoreID[0]
            CoreIndexed_PatientMap_df.at[i,'PerPatientTMAPatientID']=PerPatientTMAPatientID[0]
        elif i ==691:
            CoreIndexed_PatientMap_df.at[i,'LNM Label']=Label[1]
            CoreIndexed_PatientMap_df.at[i,'PerPatientTMACoreID']=PerPatientTMACoreID[1]
            CoreIndexed_PatientMap_df.at[i,'PerPatientTMAPatientID']=PerPatientTMAPatientID[1]
CoreIndexed_PatientMap_df=CoreIndexed_PatientMap_df[CoreIndexed_PatientMap_df['LNM Label']!='']


In [7]:
#4. get list of imzML files 
#note that the numbers in the files are for *cores* not patients!!!
imzML_df = pd.DataFrame()
CC=0
Index=0
for Path in SourceData_df['TMA_path']:
    Files = glob.glob(Path + '*.imzML')
    ThisTMA = SourceData_df['TMA_ID'].values[CC]
    CC=CC+1
    for File in Files:
        imzml_name = os.path.basename(File)
        FileCoreID = re.findall('\d+', imzml_name )[0]
        #print(File,FileCoreID)
        try:
            label = CoreIndexed_PatientMap_df.loc[int(FileCoreID)]['LNM Label']
            Index+=1
            imzML_df.at[Index,'TMA_ID'] = ThisTMA
            imzML_df.at[Index,'label'] = label
            imzML_df.at[Index,'File'] = File
            imzML_df.at[Index,'imzml_name'] = imzml_name
            imzML_df.at[Index,'core ID'] = FileCoreID
            imzML_df.at[Index,'patient'] = CoreIndexed_PatientMap_df.loc[int(FileCoreID)]['PerPatientTMAPatientID']
        except:
            #if we're here, its because we've excluded patients for which the label was "not tested" or n/a
            pass
imzML_df['patient']=imzML_df['patient'].astype('int')
imzML_df=imzML_df.sort_values(by=['patient'])

In [8]:
imzML_df.drop_duplicates(subset=['patient'])['label'].value_counts()

negative    209
positive     42
Name: label, dtype: int64

In [9]:
#6. get the first and last mz points for all per-patient data and find the maximum and minimum mzs bin in all the data
def ProcessFile(FileName,SavePath,DataKind,TMA_ID,PatientID,DoSave):
    p = ImzMLParser(FileName)
    mzs, intens = p.getspectrum(0)
    mzs = np.zeros(len(intens),'float32')
    intensities = np.zeros((len(p.coordinates),len(intens)),'float32')
    coords = np.zeros((len(p.coordinates),3),'float32')
    for idx, (x,y,z) in enumerate(p.coordinates):
        #assume mzs same for all
        mzs,intensities[idx,:] = p.getspectrum(idx)
        coords[idx,:] = [x,y,z]
    first_point = mzs[0]
    last_point = mzs[-1]
    OutFileName = SavePath+PatientID + '_' + DataKind + '_' + TMA_ID+ '.mat'
    if DoSave == True:
        sio.savemat(OutFileName, {'mzs':mzs,'coords':coords,'intensities':intensities})
    return intensities.shape[1],first_point,last_point
#get the first and last mz points
start = time.time()
for index, row in imzML_df.iterrows():
    if row['File'] != None:
        this_len,first_point,last_point = ProcessFile(row['File'],'Tuebingen/','All',row['TMA_ID'],str(index),False)
        imzML_df.at[index,'num_mzs_points']=this_len
        imzML_df.at[index,'first_mzs_val']=first_point
        imzML_df.at[index,'last_mzs_val']=last_point
end = time.time()
print(end - start)
#get the maximum and minimum bin
Overall_min_mzs = min(imzML_df['first_mzs_val'])
Overall_max_mzs = max(imzML_df['last_mzs_val'])
print(Overall_min_mzs,Overall_max_mzs)


C:\Users\Mark\Anaconda3\envs\MALDI_ML\lib\site-packages\pyimzml\ontology\ontology.py:94: UserWarning: Accession IMS:1000046 found with incorrect name "pixel size". Updating name to "pixel size (x)".
  % (accession, raw_name, name)


32.51710605621338
799.8306884765625 4496.33837890625


In [10]:
#8. functions for binning mzs data to reduce size
def GetBinnedData(BinEdges,FileName):
    p = ImzMLParser(FileName)
    mzs, intens = p.getspectrum(0)
    mzs = np.zeros(len(intens),'float32')
    Binned_intensities = np.zeros((len(p.coordinates),len(BinEdges)),'float32')
    coords = np.zeros((len(p.coordinates),3),'float32')
    for idx, (x,y,z) in enumerate(p.coordinates):
        #assume mzs same for all
        mzs,intensities = p.getspectrum(idx)
        Mzs_binning_indices = np.digitize(mzs, BinEdges)
        Binned_intensities[idx,:] = [intensities[Mzs_binning_indices == i].sum() for i in range(1, len(BinEdges)+1)]
        coords[idx,:] = [x,y,z]
    return mzs,Binned_intensities
def BinSpectra(BinEdges,data_df):
    data_df=data_df.astype(object)
    start = time.time()
    for index, row in data_df.iterrows():
        print(index)
        if row['File'] != None:
            this_mzs,this_intensities = GetBinnedData(BinEdges,row['File'])
            data_df.at[index,'binned_data'] =''
            data_df=data_df.astype(object)
            data_df.at[index,'binned_data'] = [this_intensities]
        end = time.time()
        print(end - start)
    return data_df

In [11]:
#run time for this cell much slower for smaller dz
dz=3

#get the boundaries of the bins
BinEdges = np.linspace(Overall_min_mzs-dz,Overall_max_mzs,int((Overall_max_mzs-Overall_min_mzs)/dz))

#bin the TMA data
Data_df = BinSpectra(BinEdges,imzML_df)



1
11.007559061050415
2
29.054346561431885
3
36.999135971069336
4
55.224472999572754
5
67.37497353553772
6
79.73299169540405
7
94.17646479606628
8
108.08127284049988
9
124.96910214424133
10
152.66402506828308
11
177.16548991203308
12
198.51538372039795
13
222.62290215492249
14
239.5516219139099
15
256.6698348522186
16
266.83065724372864
17
281.62812328338623
18
304.07009744644165
19
326.4801559448242
20
349.4337592124939
22
352.7977614402771
21
367.0915312767029
23
384.82609510421753
24
414.528648853302
25
424.5687937736511
26
444.54245114326477
27
473.51899671554565
28
490.4257752895355
29
516.0233571529388
30
526.4215452671051
31
546.7611391544342
32
565.4720931053162
33
587.506418466568
34
610.0726902484894
35
622.9233176708221
36
650.0387904644012
37
665.7387959957123
38
680.7346842288971
39
687.2653238773346
43
714.9024012088776
42
740.6365690231323
41
760.2201874256134
40
782.3430120944977
44
810.323225736618
45
837.0906774997711
46
838.8739078044891
47
843.5244705677032
48
853.73

In [13]:
#save as a h5 file
Data_df.to_hdf('BinnedData/Tuebingen_Jan2021_dz'+str(dz)+'_df.h5', key='Tuebingen_Jan2021_dz', mode='w')
    

C:\Users\Mark\Anaconda3\envs\MALDI_ML\lib\site-packages\pandas\core\generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['TMA_ID', 'label', 'File', 'imzml_name', 'core ID', 'patient',
       'num_mzs_points', 'first_mzs_val', 'last_mzs_val', 'binned_data'],
      dtype='object')]

  encoding=encoding,
